# Pytorch Function Showcase

In [1]:
import torch as tor
import numpy as np
import inspect

## Tensor

Definition: Tensor is a multi-dimentional array of numerical values, denoted as $k^{th}\ order\ tensor$. By default, new tensors are stored in memory for CPU-based computation.

In [7]:
# Example

# Populate a tensor representing range
x = tor.arange(10, dtype=tor.float64)

# Get #elements
print(x.numel())

# Get shape
print(x.shape)

# Reshape
print(x.reshape(2,5))

# Reshape with dimension inference
print(x.reshape(2, -1))

10
torch.Size([10])
tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]], dtype=torch.float64)
tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]], dtype=torch.float64)


### Indexing & Computation
- Similar to Numpy

### Memory Handeling
Machine learning compuation tend to be memory heavy. By defualt, reusing the same variable name cause extra memory allocation:

In [8]:
X = tor.ones(10)
Y = tor.zeros((2,1))

before = id(X)
X = X+Y
assert id(X) == before

AssertionError: 

Solution: in-place assignmnet

In [ ]:
before = id(X)
X[:] = X+Y # or X += Y
assert before == id(X)

### Conversion to Other Class

In [ ]:
# Convert to numpy array
X.numpy()

# Convert from numpy array
tor.from_numpy(np.zeros(10))

# Convert to Python scalar
X[0,0].item()

1.0

## Auto Differentiation


In [20]:
X = tor.ones((3,1))
Y = tor.zeros((3,1))

w = tor.tensor([1.], requires_grad=True)
b = tor.tensor([2.], requires_grad=True)

loss = tor.norm(X@w.reshape(-1,1)+b-Y) ** 2
loss.backward()
print(f"w gradient: {w.grad}, b gradient {b.grad}")

w gradient: tensor([18.]), b gradient tensor([18.])


In [30]:
(2 * X.T @ (X@w+b-Y)).mean()

tensor(18., grad_fn=<MeanBackward0>)

## Training NN in Jupyter Notebook: An OOP Approach
Code implementation in Python can be complex and overtly long. By convention, a NN project code base is divided into **3 modules**: `Module` class contains models, losses and optimization methods; `Data-Module` contains data loaders for training and validation; the former two classes are combined into the `Trainer` module to train on different platforms.

### Dynamic Attribute Insertion
- Allows afterwards method definition

In [ ]:
def add_to_class(Class):
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

class A:
    def __init__(self):
        self.a = 1

a = A()

@add_to_class(A)
def say_a(self): print(f'My a is {self.a}')

a.say_a()

My a is 1


### Hyperparameter Auto-saving
-  Save all __init__ parameter as class attributes

In [5]:
class HyperPrameters:
    def save_hyperparameters(self, ignore=[]):
        '''
        This function saves the arguments of the last frame as the attributes of this instance
        '''
        frame = inspect.currentframe().f_back # access the frame of last function call
        _, _, _, local_vars = inspect.getargvalues(frame)
        for k, v in local_vars.items():
            if k not in ignore: setattr(self, k, v)

Example:

In [6]:
class A(HyperPrameters):
    def __init__(self, a: int):
        self.save_hyperparameters()

a = A(10)
print(a.a)

10


### Progress Board
- Diaplay a live animation showing training progress

In [11]:
class ProgressBoard(HyperPrameters):
    def __init__(self, xlabel=None, ylabel=None, xlim=None, ylim=None, xscale='linear', yscale='linear'):
        self.save_hyperparameters()
    def draw(self, x, y, label, every_n=1):
        

### Module Class
The `module` class is thte base class of all models. Three methods need to be defined at a minimum:
- `__init__` method stores the learnable parameters
- `training_step` method accetps a data batch to return the loss value
- `configure_optimizers` method returns the optimization method(s) used to update the weights

In [12]:
from torch import nn

class Module(nn.Module, HyperPrameters):
    def __init__(self, plot_train_per_epoch=2, plot_vlaid_per_epoch=1):
        super().__init__()
        self.save_hyperparameters()
        self.board = ProgressBoard()
    
    def loss(self, y_hat, y):
        raise NotImplementedError
    
    def forward(self, X):
        assert hasattr(self, 'net')
        return self.net(X)
        
    def training_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('loss', l, train=True)
    
    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1], batch[-1]))
        self.plot('loss', l, train=False)
    
    def configure_optimizer(self):
        raise NotImplementedError

### Data
The `DataModule` class is the base class for data. The `__init__` method prepares the data, including downloading and preprocessing. It serves as an interface between trainer and dataload, a Python generator that yields data batches.

In [13]:
class DataModule(HyperPrameters):
    def __init__(self, root='./data', num_workers=4):
        self.save_hyperparameters()
    
    def get_dataloader(self, train):
        raise NotImplementedError

    def train_dataloader(self):
        return self.get_dataloader(train=True)
    
    def val_dataloader(self):
        return self.get_dataloader(train=False)

### Trainer Class
The `Trainer` class trains the network in the `Module` class with the data specified in `DataModule`. The `key` method accepts an instance of `Module` class and an instance of `DataModule` class. It iterates over the entire dataset max_epochs times to train the model.

In [14]:
class Trainer(HyperPrameters):
    def __init__(self, max_epochs: int, num_gpus=0, gradient_clip_val=0):
        self.save_hyperparameters()
        assert num_gpus == 0
    
    def prepare_data(self, data: DataModule):
        self.train_dataloader = data.train_dataloader()
        self.val_dataloader = data.val_dataloader()
        self.num_train_batches = len(self.train_dataloader)
        self.num_val_batches = len(self.val_dataloader if self.val_dataloader is not None else 0)

    def prepare_model(self, model: Module):
        model.trainer = self,
        model.board.xlim = [0, self.max_epochs]
        self.model = model
    
    def fit(self, model: Module, data: DataModule):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = model.configure_optimizers()
        self.epoch = 0
        self.train_batch_idx = 0
        self.val_batch_idx = 0
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()
    
    def fit_epoch(self):
        raise NotImplementedError

The codes above are saved in the "dl" module for modular design of the network. 